# Seperating ingredients into unit, quantity and item

1. unpack list into items
2. for each item:
3.     check taste profile csv for existence, also use NER entries. For which ingredient to use: find shortest
4.     if exists: split by " ", 1st is quantity, 2nd is unit, concat rest into item(long), taste profile title for item(short)
5.     put all these into new lists and then DF for new CSV


~30k recipes do not have the same number of ingredients and NER (extracted short ingredients). Decision: drop them.
We only want recipes with multiple valid ingredients.
Valid ingredients have a parsable unit and/or quantity.
Parsable units are units in one of the imperial or metric units defined in the conversion dictionary.
Valid quantities are quantities that can be converted to a number using our custom parsing function

In [1]:
#importing required libraries
import pandas as pd
import time

In [2]:
#reading the .csv files (use the first thenth for a quicker test run)
recipes = pd.read_csv("NLG Recipe dataset/dataset/full_dataset.csv")   #first_tenth_recipes.csv
ingredients = pd.read_csv("1683806651775-Taste,_Fat_and_Texture_Da/original/taste_profiles_V7.csv")

In [1]:
#recipes.head()

In [2]:
#ingredients.head()

In [6]:
#function used to correctly parse the lists in the ingredients column as lists
def strToLst(item):
    "Takes a string that encodes a list. Converts it into a proper list"
    return item[2:-2].split("\", \"")

In [7]:
#making a mask to remove recipes with different ingredients vs NER:
mask = [ len(strToLst(recipes["ingredients"][i])) == len(strToLst(recipes["NER"][i])) for i in range(len(recipes.values))]
recipes2 = recipes[mask]

In [8]:
recipes2.reset_index(inplace=True, drop=True)
recipes2.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
4,5,Cheeseburger Potato Soup,"[""6 baking potatoes"", ""1 lb. of extra lean gro...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"[""baking potatoes"", ""extra lean ground beef"", ..."


In [9]:
# #confirm no more duplicates
# val = 0
# for i in range(len(recipes2.values)):
#     if len(strToLst(recipes2["ingredients"][i])) != len(strToLst(recipes2["NER"][i])):
#         val += 1
# print(val)

In [11]:
#dictionary mapping diverse units to standardized ones (smostly US customary or metric ones, with all the same spelling)
#used to have only actual units in the units
unit_conversion_dict = {
    " " : " ",
    "bag" : "bags",
    "bar" : "bar",
    "bowl" : "bowls",
    "bowls" : "bowls",
    "bunch" : "bunches",
    "bunches" : 'bunches',
    "bundle" : "bunches", #only used for some spinach
    "bushel" : "bushel",
    "bxs" : "boxes",
    "c" : "cups",
    "c." : "cups",
    "c," : "cups",
    "c.c" : "cups",
    "c/" : "cups",
    "cup," : "cups",
    "cuppa" : "cups",
    "cups." : "cups",
    "cups" : "cups",
    "dash" : "dashes",
    "dashes" : "dashes",
    "dl" : "dl",
    "dot" : "dots",
    "dots" : "dots",
    "doz." : "dozen",
    "fl." : "fluid ounces",  #logical extrapolation
    "g" : "grams",
    "g." : "grams",
    "gal" : "gallons",
    "gal." : "gallons",
    "gal.)" : "gallons",
    "gallon" : "gallons",
    "gm." : "grams",
    "gr" : "grams",
    "gr." : "grams",
    "grams" : "grams",
    "head" : "heads",
    "heads" : "heads",
    "in." : "inches",
    "inch" : "inches",
    "inches" : "inches",
    "jar" : "jars",
    "jars" : "jars",
    "k." : "kilograms",
    "keg" : "jars", #there are kegs of ketchup... basically jars right?
    "kg" : "kilograms",
    "kg." : "kilograms",
    "kilo" : "kilograms",
    "kilograms" : "kilograms",
    "l.??" : "liters",
    "liters" : "liters",
    "lb" : "lbs",
    "lb." : "lbs",
    "lb)" : "lbs",
    "lb," : "lbs",
    "lb.)" : "lbs",
    "lb.)," : 'lbs',
    "lb.," : "lbs",
    "lbs" : "lbs",
    "lbs." : "lbs",
    "lbs.)" : "lbs",
    
    "liter)" : "liter",
    "liter" : "liter",
    "ml" : "ml",
    "ml." : "ml", #milliliter
    "ounce" : "ounces",
    "ounce)" : "ounces",
    "ounces" : "ounces",
    "oz" : "ounces",
    "oz)" : "ounces",
    "oz." : "ounces",
    "oz.)" : "ounces",
    "oz.)(" : "ounces",
    "oz.)." : "ounces",
    "oz.)/tcan" : "ounces",
    "no" : "nr",
    "no." : "nr",
    "package" : "packages",
    "packages" : "packages",
    "pcs." : "pieces",
    "pint" : "pints",
    "pkg" : "packages",
    "pkg." : "packages",
    "pkg.)" : "packages",
    "pkgs" : "packages",
    "pkgs." : "packages",
    "pound" : "lbs",
    "pounds" : "lbs",
    "pound)" : "lbs",
    "qt" : "quarts",
    "qt." : "quarts",
    "qt.)" : "quarts",
    "qts" : "quarts",
    "qts." : "quarts",
    "spoon" : "tablespoons", 
    "stick" : "gill", #default butter stick is 8 tablespoons which is 1 gill according to some us guy i asked
    "stick)" : "gill",
    "stick," : "gill",
    "stick?" : "gill",
    "sticks" : "gill",
    "stick)" : "gill",
    "t" : "teaspoons",
    "t." : "teaspoons",
    "tablespoon" : "tablespoons",
    "tablespoons" : "tablespoons",
    "tbls" : "tablespoons",
    "tbls." : "tablespoons",
    "tblsp." : "tablespoons",
    "tbs" : "tablespoons",
    "tbsp" : "tablespoons",
    "tbsp." : "tablespoons",
    "tbsp.)" : "tablespoons",
    "tbsp.," : "tablespoons",
    "tbsp.?" : "tablespoons",
    "tbsps." : "tablespoons",
    "teaspoon" : "teaspoons",
    "teaspoons" : "teaspoons",
    "tsp" : "teaspoons",
    "tsp." : "teaspoons",
    "tsp.)" : "teaspoons",
    "tsp.white" : "teaspoons",
    "tsps." : "teaspoons",
    "tub" : "packages",
}

In [12]:
def qtyToFloat(qty : str) -> float:
    """
    Converts a recipe quantity into a normal float.
    Should handle: 
    integers: '1'
    fractions: '1/2'
    ranges: '3-5'
    """
    
    if qty == "" or qty == " ":
        #prevent division by 0 in weird fractions (eg. '1/ '), at cost of weird values
        return 0.01
    
    nr = 0.0
    
    for i in qty.split(" "):
        #skip items with brackets (these are typically pan measurements or new recipe quantities for more/less people)
        if "(" in i or ")" in i:
            continue
        #skip items that contain letters (typically units or pan sizes)
        skip = False
        for char in i:
            if char not in "1234567890/- ":
                skip = True
        if skip:
            continue
        try:
            #integer or good float handling
            nr += abs(float(i))
        except:
            #ranges: pick smaller value
            if "-" in i:
                j = i.split("-")
                nr += qtyToFloat(j[0])
            #fractions
            elif "/" in i:
                j = i.split("/")
                nr += qtyToFloat(j[0])/qtyToFloat(j[1])
    return(nr)
            
    

In [13]:
start = time.time()
#now match ingredients for each recipe,
#with the caveat I only want to save recipes that have at least 2 associated records
records_list = []
for i in range(len(recipes2.values)):
    NER = strToLst(recipes2["NER"][i])
    items = strToLst(recipes2["ingredients"][i])
    records = []
    for j in range(len(NER)):
        val = NER[j]
        #find candidate ingredients
        lst = []
        for val2 in ingredients["Key_ingredient"]:
            if val in val2.split(" "):
                #.split increases speed of matching, but breaks it in rare case that val is multiple words
                #print("hello", val, val2)
                lst.append(val2)
        #choose final ingredient:
        if lst:
            minlen = 99999
            for val2 in lst:
                if len(val2) < minlen:
                    minlen = len(val2)
                    shortest = val2
            #print(shortest, val)
            #generate new listing: recipe, quantity, unit, ingredientNER, ingredientIngredients
            item_split = items[j].split(" ")
            qty = ""
            unit = ""
            rest = ""
            no_qty=False
            for a in item_split:
                found = False
                for k in a:
                    if k in "123456789":
                        found = True
                        break
                if found and not no_qty:
                    #assume anything containing a number is part of the quantity
                    qty += f" {a}"
                elif unit == "" and qty != "":
                    #assume right after that is a 1 word unit
                    #unless the next word has too much overlap with the NER ingredient (eg. "onion," and "onion")
                    if not a in val and not val in a:
                        unit = a
                    else:
                        unit = " "
                        rest += f" {a}"
                    #don't allow more quantities now
                    no_qty = True
                else:
                    #and anything else is rest
                    rest += f" {a}"
            #don't append records with no quantity nor unit
            if unit != " " and qty != "":
                #convert unit to standardized and quantity to a float
                new_qty = qtyToFloat(qty)
                try:
                    new_unit = unit_conversion_dict[unit.lower()]
                    record = [i, new_qty, new_unit, val, shortest, rest]
                    records.append(record)
                except:
                    pass
    #only keep recipes with at least 2 ingredients
    if len(records) >= 2:
        for record in records:
            records_list.append(record)
            
    if i % 10000 == 0:
        print(f"records: {i}, time: {time.time()-start:.2f}")
print(time.time()-start)

records: 0, time: 0.69
records: 10000, time: 3.59
records: 20000, time: 6.53
records: 30000, time: 9.42
records: 40000, time: 12.32
records: 50000, time: 15.32
records: 60000, time: 18.30
records: 70000, time: 21.36
records: 80000, time: 24.45
records: 90000, time: 27.43
records: 100000, time: 30.54
records: 110000, time: 33.71
records: 120000, time: 36.93
records: 130000, time: 39.91
records: 140000, time: 43.06
records: 150000, time: 46.19
records: 160000, time: 49.50
records: 170000, time: 52.82
records: 180000, time: 55.85
records: 190000, time: 58.96
records: 200000, time: 62.23
records: 210000, time: 65.40
records: 220000, time: 68.25
records: 230000, time: 71.00
records: 240000, time: 73.80
records: 250000, time: 76.89
records: 260000, time: 79.83
records: 270000, time: 82.77
records: 280000, time: 85.82
records: 290000, time: 88.65
records: 300000, time: 91.57
records: 310000, time: 94.52
records: 320000, time: 97.56
records: 330000, time: 100.40
records: 340000, time: 103.55
r

In [16]:
#converting results to df
new_df = pd.DataFrame(records_list)

In [17]:
print(len(new_df)) #28k ingredients    #with full matching: 85k, but roughly half are buttermilk so we don't do that

#full dataset: 180k ingredients

179466

In [18]:
#setting the df columns to desired names
new_df.columns = ["recipeID", "quantity", "unit", "ingredient_NER", "ingredient_Key_ingredient", "rest of ingredient"]
new_df.head()

,recipeID,quantity,unit,ingredient_NER,ingredient_Key_ingredient,rest of ingredient
0,5,1.00,cups,buttermilk,buttermilk,buttermilk
1,5,1.00,cups,buttermilk,buttermilk,buttermilk
2,40,0.25,cups,onion,onion,chopped onion (or as much as you want)
3,40,1.00,packages,broccoli,broccoli,chopped broccoli
4,44,4.00,cups,cabbage,cabbage,shredded cabbage


In [19]:
len(new_df["recipeID"].unique()) #110k recipes left #down to 85k now #13k    #with full matching 37k
#full dataset: 80k recipes

81923

In [20]:
len(new_df["ingredient_Key_ingredient"].unique())

#full dataset: using 73/90 ingredients

73

In [21]:
#filter recipes so we only have "useful" recipeIDs
recipes2.rename({'Unnamed: 0' : "recipeID"}, inplace=True, axis=1)

ids = list(new_df["recipeID"].unique())
mask = [recipes2["recipeID"][i] in ids for i in range(len(recipes2))]
sum(mask)

C:\Users\HP\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


67528

In [22]:
recipes3=recipes2[mask]

In [23]:
#new_df[1000:1050]

In [25]:
#save the resulting smaller recipe df (using only recipes of which we have records) and the matching df
# recipes3.to_csv("NLG Recipe dataset/dataset/cleaned_recipesv3.csv", index=False)
# new_df.to_csv("NLG Recipe dataset/dataset/recipe_ingredientsv7.csv", index=False)

In [255]:
#printing some statistics about the unique ingredients, units and quantities
print(len(new_df["unit"].unique()))

In [257]:

print(len(new_df["quantity"].unique()))

122


In [6]:
sorted(ingredients["Key_ingredient"].unique())

['apple',
 'bacon',
 'banana',
 'beef',
 'biscuit',
 'boursin',
 'bread',
 'broccoli',
 'buttermilk',
 'cabbage',
 'carrot',
 'cauliflower',
 'celeriac',
 'cereal',
 'cheese',
 'chicken',
 'chicory',
 'chips',
 'cod',
 'cooking',
 'courgette',
 'cracker',
 'croquette',
 'cucumber',
 'custard',
 'date',
 'doughnut',
 'duck',
 'egg',
 'endive',
 'gherkin',
 'ham',
 'hamburger',
 'herring',
 'honey',
 'jam',
 'kale',
 'kiwi',
 'leek',
 'lettuce',
 'liquorice',
 'luncheon',
 'macaroni',
 'mackerel',
 'marmite',
 'mayonnaise',
 'mozzarella',
 'muesli',
 'mushroom',
 'mustard',
 'olive',
 'onion',
 'orange',
 'pate',
 'pea',
 'peach',
 'peanut',
 'pear',
 'pesto',
 'pineapple',
 'popcorn',
 'pork',
 'porridge',
 'potato',
 'pretzel',
 'pumpkin',
 'radish',
 'raisin',
 'salami',
 'salmon',
 'sauce',
 'sauerkraut',
 'sausage',
 'seaweed',
 'shoarma',
 'shrimp',
 'sparerib',
 'speculaas',
 'spinach',
 'steak',
 'strawberry',
 'stroopwafel',
 'tartare',
 'tilapia',
 'toffee',
 'tomato',
 'tortil